# Praktikum 5 
- Text und Webmining WS 22/23 
- Prof. Dr. Markus Döhring, Lars Neumann 

Dieses Notebook stellt das fünfte und letzte Praktikum der Lehrveranstaltung dar. Die Aufgaben sollen alle vor Ort bearbeitet und besprochen werden.

Ziel des Praktikums ist es den Prozess der Analyse und Modelerstellung bei sequentiellen Daten am Beispiel darzustellen.

---

<a id="toc"></a>
## 📄 Inhaltverzeichnis

*In Jupyter Lab kann in der linken Seitenleiste ebenfalls ein Inhaltverzeichnis verwendet werden.*

- [Setup](#setup) 
    - [Imports](#imports)
    - [Utility Code](#util)
- [Parameter](#parameter) 
- [Teil 1: IMDB Datensatz](#dataset) 
    - [Explorative Datenanalyse](#eda)
- [Teil 2: Erstellen und Training von Modellen](#models) 
- [Klassische Ansätze](#classics) 
    - [Logistische Regression](#logistic_regression) 
    - [Decision Trees](#decision_tree) 
- [Deep Learning](#deep_learning) 
    - [Optimizer, Fehlerfunktion und Metriken](#optimizer) 
    - [RNNs mit Embeddings](#rnn)
    - [RNNs GloVE Embeddings](#rnn_glove) 
    - [LSTMs](#lstm)
    - [Transformer / Bert](#bert) 

## 📋 Aufgaben 

#### **Aufgabe 1** Datenexploration und klassische Ansätze
- [a. Mit dem Datensatz vertraut machen](#a1_a)  
- [b. Feature-Extraction](#a1_b)  
- [c. Accuracy als Metrik?](#a1_c)  
- [d. Interpretation der klassischen Modelle](#a1_d) 
#### **Aufgabe 2** Deep Learning: RNN & LSTM
- [a. Interpretation des RNN](#a2_a)  
- [b. Fehlvorhersagen betrachten](#a2_b)  
- [c. Interpretation RNN mit GloVe Embeddings](#a2_c) 
- [d. Interpretation LSTM](#a2_d) 
#### **Aufgabe 3** Deep Learning: BERT/Transformer
- [a. Interpretation BERT](#a3_a)
- [b. BertViz](#a3_b)   
- [c. Fehlvorhersagen betrachten und vergleichen](#a3_c) 
    
    

<a id='setup'></a>
# 🛠️ Setup 


Wenn dieses Notebook als *'normales' Jupyter Notebook* ausgeführt wird gehen Sie bitten in der Menüleiste oben links auf den Reiter 'File' und dann auf 'Trust this Notebook'.\
In dem Pop-up Fenster müssen Sie dann nochmal bestätigen, dass Sie diesem Notebook vertrauen. Das ist notwendig um einige ineraktive Features dieses Notebooks verwenden zu können. 

In *Jupyter Lab* ist dies nicht notwendig. 

In [ ]:
from IPython.display import HTML, Javascript, clear_output

!apt-get update
!apt install graphviz --fix-missing -y

import sys

!{sys.executable} -m pip install --upgrade pip
#!{sys.executable} -m pip install -r requirements.txt
!{sys.executable} -m pip install jupyter_black bertviz transformers datasets seaborn scikit-learn pydotplus tqdm matplotlib

clear_output()
print("Installation complete!")

# In Jupyter Notebook the kernel can be restarted via javascript
# for Jupyter Lab the following line should be commented out
Javascript("alert('Bitte Kernel neustarten!')")
# Javascript("Jupyter.notebook.restart_kernel();")

> ❗  **Neustart** \
> Sobald die Installation abgeschlossen ist:
> 1. Gehen sie bitte oben in der Taskleiste auf 'Kernel'
> 2. Klicken auf 'Restart Kernel'
>
> Danach muss die vorangehende Zelle nicht erneut ausgeführt werden. 

<a id="imports"></a>
### Imports

🔝 [Zurück zum Anfang](#toc)

In [ ]:
import jupyter_black

jupyter_black.load(lab=True)

# vis
from ipywidgets import interactive
import ipywidgets as widgets 
from IPython.display import (
    Markdown,
    HTML,
    display_markdown,
    display_html,
    display,
    Image,
    Javascript,
)
import matplotlib.pyplot as plt
from tqdm import tqdm
import pydotplus
import seaborn as sns

# i/o and system
import gc 
import subprocess
import urllib
from zipfile import ZipFile
import os
from io import StringIO
import pickle
import joblib  # pickle for pipelines

# typing
from typing import Dict, Union, Any
from typing import List, Tuple

# deep learning
import numpy as np
import tensorflow as tf
import pandas as pd

# import RNN
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, LSTM

# classic models
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz

# huggingface
from datasets import Dataset
from transformers import AutoTokenizer

# custom utility functions / classes import 
from p5_util import * 


assert (
    len(tf.config.list_physical_devices()) > 1
), "At least one cpu and one gpu should be available"


# checking reproducability / determinism 
# for each training the optimizer has the be initiliazed 
# otherwise the training isn't deterministic

RANDOM_STATE = 42
os.environ['PYTHONHASHSEED'] = '0'

tf.keras.utils.set_random_seed(RANDOM_STATE)
tf.config.experimental.enable_op_determinism()
# alternative: 
#os.environ['TF_DETERMINISTIC_OPS'] = '1'

assert tf.random.uniform([1]).numpy() == 0.6645621, """
The tensorflow seeded random number doesn't match, reproducability might not be given!"""
assert np.random.uniform() == 0.3745401188473625, """
The numpy seeded random number doesn't match, reproducability might not be given!"""

<a id="util"></a>
## Utility Code 

> In diesem Notebook werden einige Hilfsfunktionen aus der Datei `p5_util.py` genutzt. 
> Es ist nicht notwendig für die Durchführung dieses Praktikums, wer sich trotzdem einige dieser Funktionen anschauen möchte kann dies gerne über den Jupyter Dateiexplorer tun *(alternativ können sie [hier klicken](p5_util.py) um den Code zu öffnen)*.
> 
> Allgemein kann für jede Funktion mit `Shift` + `Tab` die in einer Funktion enthaltene Dokumentation angezeigt werden. Dies kann z.B. auch bei der Verwendung von Funktionen aus NumPy und anderen Bibliotheken hilfreich sein.   
> 
> 

<a id='parameter'></a>
# ⚙️ Parameter
- **VOCAB_SIZE:** Gibt die Anzahl an unqiue Wörtern im Datensatz an. Dabei werden die $n$ Wörter behalten, die am häufigsten im Trainingsdatensatz vorkommen. Die weiteren Wörter werden durch einen oov_char (<u>o</u>ut <u>o</u>f <u>v</u>ocabulary) ersetzt. Dieser wird oft auch als "UNK" oder "\<unk>" (für unknown) betitelt.
- **SEQ_LEN:** Gibt die Anzahl an Wörtern an, auf die jede Input-Sequenz in der Vektordarstellung erweitert (padding) oder gekürzt wird, um einen einheitlichen Input für das Modell zu erhalten. Beim padding wird ein spezielles Zeichen eingefügt, das dem Model signalisiert soll, welche Bedeutung diese Zeichen haben. In der Integer-Darstellung werden diese Zeichen im folgenden durch 0 repräsentiert (manchmal auch "PAD" oder "\<pad>" genannt). *Anmerkung: Nicht jedes Model benötigt dieses Datenformat.*
- **CACHED**: Wenn True, werden viele Teile des Codes nicht neu ausgeführt, sondern aus Ergebnisdateien geladen. 


In [ ]:
SEQ_LEN = 250
VOCAB_SIZE = 10000
CACHED = False


def set_values_callback(seq_dropdown, vocab_dropdown, cached_checkbox):
    def fn(event):
        global SEQ_LEN
        global VOCAB_SIZE
        global CACHED
        SEQ_LEN = int(seq_dropdown.value)
        VOCAB_SIZE = int(vocab_dropdown.value)
        CACHED = cached_checkbox.value
        display(
            Javascript(
                "Jupyter.notebook.scroll_to_cell(Jupyter.notebook.get_selected_index()+1, 250)"
            )
        )

    return fn


create_set_value_ui(
    default_vocab_size=10_000,
    default_seq_len=250,
    default_cached=False,
    callback_fn=set_values_callback,
)

<a id='dataset'></a>
# 🗃️ IMDB Datensatz

Im folgenden wird der IMDB Movie Review Datensatz <sup>[1]</sup> geladen und für die Verwendung im Training von Modellen vorverarbeitet. Der Datensatz enthält jeweils 25.000 Retensionen als Trainings- und Testdaten. 

Um den Datensatz zu laden wird die Tensorflow Keras API verwendet. Diese gibt den Datensatz in einer Integer Repräsentation aus.<br/>
Das heißt ein Review kann z.B. so aussehen: $[23, 128, 3, 8, ...]$. <br/>
Die API liefert außerdem einen word-index, der angibt welches Wort durch welche Zahl repräsentiert wird. Um Modelle zu trainieren, die einen String-Input benötigen, müssen die Integer-Vektoren zuerst umgewandelt werden. 


<div class="alert alert-block"> 
[1]: Maas, Andrew L., Raymond E., Daly, Peter T., Pham, Dan, Huang, Andrew Y., Ng, and Christopher, Potts. "Learning Word Vectors for Sentiment Analysis." . In Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies (pp. 142–150). Association for Computational Linguistics, 2011.
<br/>   
<a href="https://ai.stanford.edu/%7Eamaas/data/sentiment/">Link zum Datensatz</a>
</div>

🔝 *[Zurück zum Anfang](#toc)*

In [ ]:
if CACHED:
    dataset = joblib.load("imdb_dataset.pkl")
    print_md("Loaded IMDb dataset.")
    (X_train, y_train), (X_test, y_test) = dataset.get_data()
    (X_train_decoded, _), (X_test_decoded, _) = dataset.get_data(True)
else:
    dataset = IMDbDataset(vocabulary_size=VOCAB_SIZE)
    (X_train, y_train), (X_test, y_test) = dataset.get_data()
    (X_train_decoded, _), (X_test_decoded, _) = dataset.get_data(True)

    # uncomment to save dataset
    # joblib.dump(dataset, "imdb_dataset.pkl")

dataset.print_metadata()

<a id='a1_a'></a>
### 📋 Aufgabe 1 a.
Machen Sie Sie sich mit dem konkreten Keras-IMDB Datensatz vertraut. Schauen Sie sich z.b. einige Review-Texte und deren Labels an. \
Stimmen Sie als Leser mit den Labels überein? Warum bzw. warum nicht?

In [ ]:
# IHR CODE HIER

*Ihre Antwort können sie hier notieren.* 

<a id='a1_b'></a>
### 📋 Aufgabe 1 b.
Identifizieren Sie die relevantesten Features (Wörter) anhand des Chi<sup>2</sup>-Wertes auf den Trainingsdaten. 


**💡 Tipp:** Nutzen Sie dazu aus sklearn: CountVectorizer und chi2, sowie aus numpy argsort.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2

# IHR CODE HIER

<a id='a1_c'></a>
### 📋 Aufgabe 1 c.
Prüfen und begründen Sie, ob „accuracy“ ein sinnvolles Gütemaß für einen Klassifikator auf diesem Datensatz ist.

$
ACC = \frac{TP + TN}{TP + TN + FP + FN}
$

In [ ]:
# IHR CODE HIER

<a id='models'></a>
# 🏋️‍♂️ Erstellen und Training von Modellen

Es werden die folgenden Modelle auf den Datensatz angewendet: 

- [Logistische Regression](#log_regression)
- [Classification Tree](#decision_tree) 
- [RNN](#rnn) 
- [RNN mit GloVe](#rnn_glove)
- [LSTM](#lstm)
- [Transformer / Bert](#bert)

*[Zurück zum Anfang](#toc)*

<a id='classics'></a>
## Klassische Ansätze

Zuerst werden klassische Machine Learning Modelle auf den Datensatz angewendet. Es gehört zu den Best Practices zuerst ein oder mehrere Baseline Modelle zum Vergleichen zu verwenden, um die Leistung der neuen Modelle in einen Kontext zu setzten bzw. zu evaluieren ob diese überhaupt bessere Ergebnisse liefern.  

<a id='log_regression'></a>
### Logistische Regression 

In [ ]:
# option for this cell
save_model = False

if CACHED:
    logistic_regression_model = joblib.load("logistic_regression_model.pkl")
    print_md("Loaded logistic_regression_model.pkl.")
else:
    # eine Pipeline erlaubt es uns mehrere Module sequentiel zu einem Modell zusammenzufügen
    logistic_regression_model = Pipeline(
        [
            ("tfidf", TfidfVectorizer(ngram_range=(1, 2))),
            ("log", LogisticRegression(random_state=RANDOM_STATE)),
        ]
    )

    logistic_regression_model.fit(X_train_decoded, y_train)

    if save_model:
        joblib.dump(
            logistic_regression_model, "logistic_regression_model.pkl", compress=1
        )

In [ ]:
# Cell-Options
save_predictions = False


# Ausgabe der Features sortiert nach ihren Koeffizienten innerhalb des LogReg Modells
print("Relevanteste Koeffizienten: ")
print(
    np.array(logistic_regression_model.named_steps["tfidf"].get_feature_names_out())[
        np.argsort(logistic_regression_model.named_steps["log"].coef_[0])
    ]
)

# Vorhersagen für den Testdatensatz erzeugen
y_pred_logistic_regression = logistic_regression_model.predict(X_test_decoded)

# Metriken zur Analyse des Modells
logistic_regression_report = classification_report(
    y_pred_logistic_regression,
    y_test,
    target_names=dataset.CLASS_NAMES,
    output_dict=True,
)

# Metriken berechnen
logistic_regression_accuracy = accuracy_score(y_test, y_pred_logistic_regression)

# Ausgabe der Metriken
visualize_classification_report(logistic_regression_report, dataset.CLASS_NAMES)
print_md("$\\textbf{Accuracy=}" + f"{logistic_regression_accuracy}$")

# Speichern (im Praktikum nicht relevant)
if save_predictions:
    joblib.dump(y_pred_logistic_regression, "y_pred_logistic_regression.pkl")

<a id='decision_tree'></a>
### Decision Tree

In [ ]:
# Cell-Options
save_model = False

# Laden oder Trainieren des Modells
if CACHED:
    decision_tree_model = joblib.load("decision_tree_model.pkl")
    print_md("Loaded decision_tree_model.pkl.")
else:
    # Sklearn Pipeline mit tf-idf und Decision Tree
    decision_tree_model = Pipeline(
        [
            ("tfidf", TfidfVectorizer(ngram_range=(1, 2))),
            (
                "tree",
                DecisionTreeClassifier(
                    criterion="gini", max_depth=4, random_state=RANDOM_STATE
                ),
            ),
        ]
    )

    # Training des Modells
    decision_tree_model.fit(X_train_decoded, y_train)

    # Speichern (im Praktikum nicht relevant)
    if save_model:
        joblib.dump(decision_tree_model, "decision_tree_model.pkl", compress=1)

In [ ]:
# Cell-Options
save_predictions = False

# Vorhersagen für den Testdatensatz erzeugen
y_pred_decision_tree = decision_tree_model.predict(X_test_decoded)

# Metriken zur Analyse des Modells
decision_tree_report = classification_report(
    y_pred_decision_tree, y_test, target_names=dataset.CLASS_NAMES, output_dict=True
)
decision_tree_accuracy = accuracy_score(y_test, y_pred_decision_tree)
visualize_classification_report(decision_tree_report, dataset.CLASS_NAMES)

# Ausgabe der Metriken
print_md("$\\textbf{Accuracy=}" + f"{decision_tree_accuracy}$")

# Speichern (im Praktikum nicht relevant)
if save_predictions:
    joblib.dump(y_pred_decision_tree, "y_pred_decision_tree.pkl")

In [ ]:
# Holt die Split-Tokens des Models
list_decision_tree_split_arguments(decision_tree_model)

In [ ]:
%matplotlib notebook
if CACHED:
    display(Image(filename="decision_tree_graph.png"))
else:
    # Falls Sie https://www.graphviz.org/ installiert haben, koennen Sie den Decision Tree auch plotten
    # In der twm VM koennen Sie graphviz mit: sudo apt-get -y install graphviz
    # installieren
    if check_graphviz_installation():
        display_decision_tree(decision_tree_model)
    else:
        print("Please install graphviz.")

<a id='a1_d'></a>
### 📋 Aufgabe 1 d.

Wie interpretieren Sie die Ergebnisse der „einfacheren“ Klassifikationsmodelle auf den Testdaten bzw. was fällt Ihnen auf? \
Inwiefern deckt sich die Wichtigkeit der Features mit der, die Sie in Teilaufgabe b ermittelt haben?

*Ihre Antwort können Sie hier notieren.*

<a id='deep_learning'></a>
# 🧠 Deep Learning

🔝 *[Zurück zum Anfang](#toc)*

### Anpassung der Trainingsdaten 
Die Inputs müssen gepaddet werden damit alle Datenpunkte eine einheitliche Länge haben.<br> 
Das heißt, dass ein kürzerer Text (in der Vektor-Repräsentation) mit Nullen aufgefüllt wird um eine bestimmte Länge zu erreichen. 
Ein Längerer Text wird abgeschnitten.
> z.B. Sequence-Length = 10: $[1,2,3,4,5,6]$ => $[1,2,3,4,5,6,0,0,0,0]$

In [ ]:
# Padding auf die oben angegebene Sequenz-Länge
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=SEQ_LEN)
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=SEQ_LEN)

# Ausgabe der shapes
print_md(
    f"""
$Pad \\space sequences \\space = \\space (samples \\times timesteps)$
- $Padded \\space shape \\space (train) = {X_train_padded.shape}$
- $Padded \\space shape \\space (test) = {X_test_padded.shape}$
"""
)

<a id="optimizer"></a>
### Optimizer, Fehlerfunktion und Metriken

Um ein Deep Learning Modell zu trainieren sind, sind so wie bei den vorangehenden klassischen Machine Learning Modellen, eine Optimierungsfunktion und eine Fehlerfunktion notwendig. Da diese beim Deep Learning häufig stark von der Architektur des Netzes und der Problemstellung abhängig sind, müssen diese mit höherer Wahrscheinlichkeit als bei z.B. einem Decision Tree angepasst werden. 

- **Adam:** Als Optimierungsalgorithmus wird Adam gewählt. Dieser zählt zu der Familie der adaptiven Optimierungsalgorithmen. Das bedeutet er kann die Schrittgröße der Fehlerrückführung selbständig anpassen und erleichtert somit die Hyperparameterwahl. [Mehr dazu](https://arxiv.org/pdf/1412.6980.pdf)
- **Binary Crossentropy:** Als Fehlerfunktion wird die Binary Crossentropy, also der Spezialfall der Kreuentropy im diskreten bei einer Ergenismenge der Mächtigkeit 2 gewählt. Dabei besteht die Möglichkeit ein Netz mit Sigmoid-Aktivierung am Ausgang zu verwenden um ide Vorhersagen auf das Intervall $[0,1]$ zu mappen. Alternativ kann das Netz darauf trainiert werden Vorhersagen im Bereich von $[-\infty, \infty]$ zu machen. In der tensorflow-Dokumentation wird empfohlen letztere Option zu wählen. [Mehr dazu](https://en.wikipedia.org/wiki/Cross_entropy)


Zusätzlich werden typischerweise noch Metriken definiert um bereits während des Traingsprozesses die Performance des Models überwachen zu können. Im folgenden wird die breits bekannte **Accuracy** verwendet.


In [ ]:
def create_compile_options():
    """Initializes the optimizer, loss, and metrics

    This can be called to assure reproducability of one or more cells.
    If more than one cell should be reproducable no other code should be executed inbetween.
    """
    global adam_optimizer
    adam_optimizer = tf.keras.optimizers.Adam()

    global bce_loss
    # from_logits=True bedeutet, dass ein Zahlenbereich von [-inf,inf] erwartet wird
    bce_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

    global binary_acc
    # Die BinaryAccuracy ist gleichbedeutend mit der normalen Accuracy für binäre Klassifikationsprobleme
    binary_acc = [tf.keras.metrics.BinaryAccuracy()]


# Init.
create_compile_options()

<a id='rnn'></a>
### Recurrent Neural Networks (RNNs) mit Embeddings

🔝 *[Zurück zum Anfang](#toc)*

RNNs sind eine Kategorie an Neuronalen Netzen, die vor der Tranformer-Architektur den Standard für die Verarbeitung von sequentiellen Daten dargestellt haben. Vereinfach kann man sich ein RNN als ein neuronales Netz vorstellen, das wie eine `for`-Schleife über die Elemente einer Sequenz iteriert. 
Diese Art der Verarbeitung ermöglicht es z.B. den Zusammenhang zwischen Wörtern an unterschiedlichen Stellen im Satz zu erkennen.

Das zentrale Problem, das alle RNN-Architekturen gemeinsam haben ist, das durch die `for`-Schleifen Verarbeitung keine Parallelisierung möglich ist und somit die Berechnung dieser Modelle Zeitintensiv ist. Weiterhin hat diese Architektur Probleme damit Zusammenhänge über längere Sequenzen zu verstehen, da mit längerer Sequenzlänge auch die Komplexität der Informationen, die zu vorherigen Elementen der Sequenz behalten werden müssen, steigt. 

https://www.tensorflow.org/guide/keras/rnn

Im Folgenden wird die die Keras-Klasse `Sequential` zu Modelerstellung verwendet. Die Klasse repäsentiert eine lineare Abfolge von Schichten (d.h. jede Schicht hat max. 1 Vorgänger bzw. Nachfolger). <br>
Bei komplexeren Modellen kann z.B. ein Graph mit parallelen Schichten erstellt werden (d.h. eine Schicht hat potentiell mehr als einen Vorgänger bzw. Nachfolger). In einem solchen Fall muss ein anderes Format gewählt werden um das Modell zu erstellen. 

Die verwendeten Schichten: 
- **Embedding:** Mapt ein als Integer dargestelltes Wort auf einen Feature-Vektor 
- **SimpleRNN:** Eine einfach sequentielle Variante der Fully Connected Schichten, die aus der Vorlesung bekannt sind. 
- **Dense:** Eine "standard" Fully Connected Schicht, die verwendet wird um den Output der RNN Schicht auf eine einzige Zahl zu reduzieren, die zur Klassifikation interpretiert werden kann.

In [ ]:
from typing import List, Union

In [ ]:
def create_rnn(
    vocab_size: int,
    optimizer: [str, tf.keras.optimizers.Optimizer],
    loss: Union[str, tf.keras.losses.Loss],
    metrics: List[Union[str, tf.keras.metrics.Metric]],
    rnn_units: int = 32,
    embedding_dim: int = 32,
):
    """
    Creates and compiles a RNN model

    Arguments:
        vocab_size (int): number of words in vocabulary
        optimizer (str | tf.keras.optimizers.Optimize): Optmizer used to compile the model
        loss (str | tf.keras.losses.Loss):
        metrics (List[str | tf.keras.metrics.Metric]):

    Creates a sequential RNN with the following architecture:
        - Embedding layer (vocab_size, embedding_dim)
        - SimpleRNN layer
        - Single neuron dense layer
    """

    # Ein Sequential Model ist eine einfache Folge von Schichten ohne Branches
    model = tf.keras.Sequential()

    # Embedding Layer: mappt Index eines Tokens auf einen Dense-Vektor
    model.add(Embedding(vocab_size, embedding_dim))

    # Eine einfache Feed-Forward RNN Schicht
    model.add(tf.keras.layers.SimpleRNN(units=rnn_units))

    # Eine Dense-Schicht mit einem Neuron ohne Aktivierungsfunktion
    model.add(Dense(1, activation=None))

    # Compile
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

In [ ]:
# options for this cell
save_history = (
    False  # should not be needed as the same prediction vector is provided anyway
)

# assure reproducability
create_compile_options()
tf.keras.utils.set_random_seed(RANDOM_STATE)

# train or load trained model
if CACHED:
    rnn_model = tf.keras.models.load_model("rnn_model_e3.h5")
    rnn_training_history = joblib.load("rnn_training_history.pkl")
    print_md("Loaded rnn model and training-history.")
else:
    # set for reproducablility
    tf.keras.utils.set_random_seed(RANDOM_STATE)

    rnn_model = create_rnn(
        vocab_size=VOCAB_SIZE,
        optimizer=adam_optimizer,
        loss=bce_loss,
        metrics=binary_acc,
    )
    rnn_training_history = rnn_model.fit(
        X_train_padded, y_train, epochs=10, batch_size=128, validation_split=0.2
    )

rnn_model.summary()

if save_history:
    joblib.dump(rnn_training_history, "rnn_training_history.pkl", compress=1)

In [ ]:
%matplotlib inline
plot_training_history(rnn_training_history)

<a id='a2_a'></a>
### 📋 Aufgabe 2 a.
Wie interpretieren Sie die Performance-Kurven und Ergebnisse auf den Testdaten zum RNN bzw. was fällt Ihnen auf?

Im folgenden wird das Model erneut auf den ganten Datensatz trainiert. <br/>
Setzten sie den angzeigten Slider auf eine angemessene Anzahl an Epochen. 

*Ihre Antwort können Sie hier notieren*

In [ ]:
rnn_epoch_interacton = interactive(lambda epochs: epochs, epochs=(1, 10, 1), value=2)
rnn_epoch_interacton.children[0].value = 2
display(rnn_epoch_interacton)

In [ ]:
# Cell-Options
save_predictions = False
save_model = False

# Reproduzierbarkeit dieser Zelle sicherstellen
create_compile_options()
tf.keras.utils.set_random_seed(RANDOM_STATE)

# Das Model für das Training auf dem ganzen Datensatz neu erstellen
rnn_model = create_rnn(VOCAB_SIZE, adam_optimizer, bce_loss, binary_acc)

print_md("**Training des Models**")
rnn_model.fit(
    X_train_padded, y_train, epochs=rnn_epoch_interacton.result, batch_size=128
)

print_md("**Evaluierung des Models auf den Testdaten**")
# Vorhersagen für den Testdatensatz erzeugen
y_pred_logits_rnn_model = rnn_model.predict(X_test_padded)
# Aktivierungsfunktion auf Logits anwenden (mapping [-inf, inf] -> [0,1])
y_pred_rnn_model = (
    tf.keras.activations.sigmoid(y_pred_logits_rnn_model).numpy().reshape(-1)
)

# Metriken zur Analyse des Modells
rnn_model_accuracy = binary_acc[0](y_test, y_pred_rnn_model)
rnn_model_loss = bce_loss(y_test.reshape(-1, 1), y_pred_logits_rnn_model)

# Ausgabe der Metriken
display_table(
    table_data=[
        ["<strong>Accuracy auf den Testdaten</strong>", "%.4f" % rnn_model_accuracy],
        [f"<strong>Loss auf den Testdaten</strong>", "%.4f" % rnn_model_loss],
    ],
    use_header=False,
)

# Speichern (im Praktikum nicht relevant)
if save_predictions:
    joblib.dump(y_pred_rnn_model.numpy(), "y_pred_rnn_model.pkl")
if save_model:
    rnn_model.save("rnn_model_e3.h5")

<a id='a2_b'></a>
### 📋 Aufgabe 2 b.

Schauen Sie sich zum RNN einige der "drastischsten" FNs an (hoher Score und Label=1). Können Sie erahnen, was das Modell ggf. verwirrt hat?

In [ ]:
# IHR CODE HIER

*Ihre Antwort können sie hier notieren*

<a id='rnn_glove'></a>
### Recurrent Neural Networks mit GloVe Embeddings(RNNs)
Im Folgenden wird die gleiche RNN-Architektur mit 50-dimensionalen GloVe Embeddings erstellt. Es wird der gleiche Optimizer, die gleiche Fehlerfunktion und die gleiche Metrik wie bei dem normalen RNN verwendet. Es werden potentiell nicht alle Best-Practices verwendet, die es zu Transfer-Learning gibt (Bsp. [Tensorflow Warm-Start Embeddings](https://www.tensorflow.org/tutorials/text/warmstart_embedding_matrix)).

*[Zurück zum Anfang](#toc)*

#### GloVe Embeddings laden

In [ ]:
# Download & laden (sollte bereits im S3 Bucket abgelegt sein)
download_glove_embeddings()
glove_word_embeddings = load_glove_embeddings()

# Es wird eine Embedding-Matrix mit den vorhandenen Tokens erstellt
# nicht gefundende Tokens werden mit 0 initialisiert
glove_weights = create_embeddings_matrix(
    vocab_size=VOCAB_SIZE,
    word_index=dataset.word_index,
    word_embeddings=glove_word_embeddings,
)

In [ ]:
type(glove_weights)

#### Model definieren

In [ ]:
def create_rnn_model_with_glove(
    glove_weights: [np.ndarray],
    optimizer: [str, tf.keras.optimizers.Optimizer],
    loss: Union[str, tf.keras.losses.Loss],
    metrics: List[Union[str, tf.keras.metrics.Metric]],
    rnn_units: int = 32,
    embedding_dim: int = 32,
):
    """
    Creates and compiles a RNN model with a given embedding matrix.

    Arguments:
        glove_weights (np.ndarray): weights for the embedding layer
        vocab_size (int): number of words in vocabulary
        optimizer (str | tf.keras.optimizers.Optimize): Optmizer used to compile the model
        loss (str | tf.keras.losses.Loss):
        metrics (List[str | tf.keras.metrics.Metric]):

    Creates a sequential RNN with the following architecture:
        - Embedding layer (vocab_size, embedding_dim)
        - SimpleRNN layer
        - Single neuron dense layer
    """
    # Ein Sequential Model ist eine einfache Folge von Schichten ohne Branches
    model = tf.keras.Sequential()

    model.add(
        Embedding(
            input_dim=VOCAB_SIZE,
            output_dim=50,
            weights=[glove_weights],
            trainable=False,
        )
    )
    # Eine einfache Feed-Forward RNN Schicht
    model.add(tf.keras.layers.SimpleRNN(units=rnn_units))

    # Eine Dense-Schicht mit einem Neuron ohne Aktivierungsfunktion
    model.add(Dense(1, activation=None))

    # Compile
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

**Modell trainieren bzw. laden**

In [ ]:
# Cell-Options
save_history = False

# Reproduzierbarkeit dieser Zelle sicherstellen
create_compile_options()
tf.keras.utils.set_random_seed(RANDOM_STATE)

# Laden oder Trainieren des Modells
if CACHED:
    rnn_glove_training_history = joblib.load("rnn_glove_training_history.pkl")
    print_md("Loaded rnn_glove training history.")
else:
    # Model erstellen
    rnn_glove_model = create_rnn_model_with_glove(
        glove_weights=glove_weights,
        optimizer=adam_optimizer,
        loss=bce_loss,
        metrics=binary_acc,
    )
    # Aufbau des Netzes anzeigen
    rnn_glove_model.summary()

    # Training des Modells
    rnn_glove_training_history = rnn_glove_model.fit(
        x=X_train_padded, y=y_train, epochs=10, batch_size=128, validation_split=0.2
    )

    # Speichern (im Praktikum nicht relevant)
    if save_history:
        joblib.dump(
            rnn_glove_training_history, "rnn_glove_training_history.pkl", compress=1
        )

%matplotlib inline
plot_training_history(rnn_glove_training_history)

**Modell auf vollständige Trainingsdaten trainieren** 

In [ ]:
# Cell-Options
save_predictions = False
save_model = False
rnn_glove_epochs = 3

# Reproduzierbarkeit dieser Zelle sicherstellen
create_compile_options()
tf.keras.utils.set_random_seed(RANDOM_STATE)

# Neues Model für Training auf den vollständigen Datensatz erstellen
rnn_glove_model = create_rnn_model_with_glove(
    glove_weights=glove_weights,
    optimizer=adam_optimizer,
    loss=bce_loss,
    metrics=binary_acc,
)

print_md("**Training des Models**")
rnn_glove_model.fit(X_train_padded, y_train, epochs=rnn_glove_epochs, batch_size=128)

print_md("**Evaluierung des Models auf den Testdaten**")
# Vorhersagen für den Testdatensatz erzeugen
# model.evaluate(...) wird nicht verwendet, um die Vorhersagen weiter analysieren zu können
y_pred_logits_rnn_glove_model = rnn_glove_model.predict(X_test_padded)
y_pred_rnn_glove_model = tf.keras.activations.sigmoid(y_pred_logits_rnn_glove_model)

# Metriken zur Analyse des Modells
rnn_glove_model_accuracy = binary_acc[0](y_test, y_pred_rnn_glove_model)
rnn_glove_model_loss = bce_loss(y_test.reshape(-1, 1), y_pred_logits_rnn_glove_model)

# Ausgabe der Metriken
display_table(
    table_data=[
        [
            "<strong>Accuracy auf den Testdaten</strong>",
            "%.4f" % rnn_glove_model_accuracy,
        ],
        [f"<strong>Loss auf den Testdaten</strong>", "%.4f" % rnn_glove_model_loss],
    ],
    use_header=False,
)

# Speichern (im Praktikum nicht relevant)
if save_model:
    rnn_glove_model.save("rnn_glove_model_e3.h5")
if save_predictions:
    joblib.dump(
        np.array(y_pred_rnn_glove_model).reshape(-1),
        "rnn_glove_training_history.pkl",
        compress=1,
    )

<a id='a2_c'></a>
### 📋 Aufgabe 2 c.

Wie interpretieren Sie die Performance-Kurven und Ergebnisse auf den Testdaten zum RNN mit vorgelernten Glove-Embedding bzw. was fällt Ihnen auf?

*Ihre Antwort können Sie hier notieren.*

<a id="lstm"></a>
### Long Short Term Memory (LSTM)

🔝 *[Zurück zum Anfang](#toc)*

**Funktion die eine Instanz des Models erstellt** 

In [ ]:
def create_lstm_model(
    optimizer: [str, tf.keras.optimizers.Optimizer],
    loss: Union[str, tf.keras.losses.Loss],
    metrics: List[Union[str, tf.keras.metrics.Metric]],
    rnn_units: int = 32,
    embedding_dim: int = 32,
):
    """
    Creates and compiles a LSTM model.

    Arguments:
        optimizer (str | tf.keras.optimizers.Optimize): Optmizer used to compile the model
        loss (str | tf.keras.losses.Loss): Loss function used to compile the model
        metrics (List[str | tf.keras.metrics.Metric]): List of metrics used to compile the model
        rnn_units (int): Number of LSTM units
        embedding_dim (int): Amount of embeddings dimensions

    Creates a sequential RNN with the following architecture:
        - Embedding layer (vocab_size, embedding_dim)
        - LSTM layer
        - Single neuron dense layer
    """

    # Sequentielles Modell erstellen
    model = Sequential()

    # Embedding Schicht für mapping von Token-Index auf Dense-Vektor
    model.add(Embedding(VOCAB_SIZE, embedding_dim))

    # Feed-Forward LSTM Schicht
    model.add(LSTM(rnn_units, name="lstm"))

    # Eine Dense-Schicht mit einem Neuron ohne Aktivierungsfunktion
    model.add(Dense(1, activation=None))

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

**Training des Models** \
*Im cached-Modus wird die Trainings-Historie geladen.*

In [ ]:
# Cell-Options
save_history = False

# Reproduzierbarkeit dieser Zelle sicherstellen
create_compile_options()
tf.keras.utils.set_random_seed(RANDOM_STATE)

# Laden oder Trainieren des Modells
if CACHED:
    lstm_training_history = joblib.load("lstm_training_history.pkl")
    print_md("Done loading lstm training-history.")

    # Plot der Trainingshistorie
    plot_training_history(lstm_training_history)
else:
    # LSTM Modell erstellen und trainieren
    lstm_model = create_lstm_model(
        optimizer=adam_optimizer, loss=bce_loss, metrics=binary_acc
    )

    # Training des Modells
    lstm_training_history = lstm_model.fit(
        X_train_padded, y_train, epochs=10, batch_size=128, validation_split=0.2
    )

    # Plot der Trainingshistorie
    plot_training_history(lstm_training_history)

# Speichern (im Praktikum nicht relevant)
if save_history:
    joblib.dump(lstm_training_history, "lstm_training_history.pkl", compress=1)

In [ ]:
# Cell-Options
save_predictions = False
save_model = False
lstm_training_epochs = 3

# Reproduzierbarkeit dieser Zelle sicherstellen
create_compile_options()
tf.keras.utils.set_random_seed(RANDOM_STATE)

# Neues LSTM Model erstellen
lstm_model = create_lstm_model(
    optimizer=adam_optimizer, loss=bce_loss, metrics=binary_acc
)

print_md("**Training des Models**")
lstm_model.fit(X_train_padded, y_train, epochs=lstm_training_epochs, batch_size=128)

print_md("**Evaluierung des Models auf den Testdaten**")

# Vorhersagen für den Testdatensatz erzeugen
y_pred_logits_lstm_model = lstm_model.predict(X_test_padded)
y_pred_lstm_model = (
    tf.keras.activations.sigmoid(y_pred_logits_lstm_model).numpy().reshape(-1)
)

# Metriken zur Analyse des Modells
lstm_model_accuracy = binary_acc[0](y_test, y_pred_lstm_model)
lstm_model_loss = bce_loss(y_test.reshape(-1, 1), y_pred_logits_lstm_model)

# Ausgabe der Metriken
display_table(
    table_data=[
        ["Metriken auf den Testdaten", "Value"],
        ["<strong>Accuracy</strong>", "%.4f" % lstm_model_accuracy],
        [f"<strong>Loss", "%.4f" % lstm_model_loss],
    ],
    use_header=True,
)

# Speichern (im Praktikum nicht relevant)
if save_model:
    lstm_model.save("lstm_model_e3.h5")

if save_predictions:
    joblib.dump(y_pred_lstm_model.numpy(), "y_pred_lstm_model.pkl")

<a id='a2_d'></a>
### 📋 Aufgabe 2 d.

Wie interpretieren Sie die Performance-Kurven und Ergebnisse auf den Testdaten zum LSTM bzw. was fällt Ihnen auf?

*Ihre Antwort können Sie hier notieren* 

<a id="bert"></a>
### BERT 

🔝 *[Zurück zum Anfang](#toc)*

**Datensatz konvertieren und tokenizen**

In [ ]:
# Reproduzierbarkeit dieser Zelle sicherstellen
tf.keras.utils.set_random_seed(RANDOM_STATE)

# Daten zu einem huggingface dataset konvertieren, um sie zu tokenizen
ds_train = Dataset.from_dict({"text": X_train_decoded, "label": y_train})
ds_test = Dataset.from_dict({"text": X_test_decoded, "label": y_test})

print("ds_train:\n", ds_train, "\n")
print("ds_test:\n", ds_test, "\n")

# Tokenizer erstellen und auf Datensatz erstellen
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Trainingsdaten tokenizen
tokenized_train_data = tokenizer(
    ds_train["text"],
    return_tensors="np",
    max_length=SEQ_LEN,
    padding="max_length",
    truncation=True,
)

# Testdaten tokenizen
tokenized_test_data = tokenizer(
    ds_test["text"],
    return_tensors="np",
    max_length=SEQ_LEN,
    padding="max_length",
    truncation=True,
)

# Konvertieren der labels (redundant)
train_labels = np.array(ds_train["label"])
test_labels = np.array(ds_test["label"])

# Tensorflow dataset für das Training erstellen
tf_ds_train = (
    tf.data.Dataset.from_tensor_slices((dict(tokenized_train_data), train_labels))
    .cache()
    .shuffle(25000)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

#### Training bzw. Finetuning des BERT-Models

Die Warnung die angibt, dass kein Loss also keine Fehlerfunktion angegeben wird, kann einfach ignoriert werden. Das Modell enthält bereits eine Fehlerfunktion im Model-Gaphen. 

In [ ]:
import transformers
from transformers import TFAutoModelForSequenceClassification
from bertviz import model_view, head_view
import torch
from transformers import logging

logging.set_verbosity_error()

bert_adam_lr = 2e-5

**Erstellen des Models** \
*Im cached-Modus wird es  geladen.*

In [ ]:
# Cell-Options
use_additional_loss_fn = True
save_model = False  # should be true for reloading without attention outputs

# Reproduzierbarkeit dieser Zelle sicherstellen
create_compile_options()
tf.keras.utils.set_random_seed(RANDOM_STATE)

# add own loss function to the output
bert_loss = bce_loss if use_additional_loss_fn else None

# Laden oder Trainieren des Models
if CACHED:
    # Laden des Models ohne attention output um OOM-Fehlern vorzubeugen
    bert_model = TFAutoModelForSequenceClassification.from_pretrained(
        "./bert_model_e2", num_labels=1, output_attentions=False
    )
    # Compile: Adam optimizer hier mit niedrigerer learning-rate
    bert_model.compile(
        optimizer=tf.keras.optimizers.Adam(bert_adam_lr),
        loss=bert_loss,
        metrics=binary_acc,
    )
    print_md("Bert Model geladen und kompiliert.")
else:
    # Distilbert Model laden
    bert_model = TFAutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased", num_labels=1, output_attentions=False
    )

    # Compile: Adam optimizer hier mit niedrigerer learning-rate
    bert_model.compile(
        optimizer=tf.keras.optimizers.Adam(bert_adam_lr),
        loss=bert_loss,
        metrics=binary_acc,
    )

    # Training des Models
    bert_history = bert_model.fit(tf_ds_train, epochs=2)

    # Speichern (im Praktikum nicht relevant)
    if save_model:
        bert_model.save_pretrained("bert_model_e2")

**Evaluierung auf Testdatensatz**

In [ ]:
# Cell-Options
save_predictions = False

# Reproduzierbarkeit dieser Zelle sicherstellen
create_compile_options()
tf.keras.utils.set_random_seed(RANDOM_STATE)

print_md("**Evaluierung des Models auf den Testdaten**")
if CACHED:
    y_pred_bert_model = joblib.load("y_pred_bert_model.pkl")
else:
    y_pred_bert_model_logits = bert_model.predict(dict(tokenized_test_data)).logits
    y_pred_bert_model = (
        tf.keras.activations.sigmoid(y_pred_bert_model_logits).numpy().reshape(-1)
    )

# Metriken zur Analyse des Models
bert_model_accuracy = tf.keras.metrics.binary_accuracy(y_test, y_pred_bert_model)
print_md(f'**Accuracy auf den Testdaten:** {"%.4f" % bert_model_accuracy}')

# Speichern (im Praktikum nicht relevant)
if save_predictions:
    joblib.dump(
        y_pred_bert_model,
        "y_pred_bert_model.pkl",
    )

<a id='a3_a'></a>
### 📋 Aufgabe 3 a.

Wie interpretieren Sie die Performance-Kurven und Ergebnisse auf den Testdaten mit BERT bzw. was fällt Ihnen auf?

*Ihre Antwort können Sie hier notieren* 

**Laden des Models mit attention-output**

In [ ]:
# reload model but with output_attention=True
create_compile_options()
tf.keras.utils.set_random_seed(RANDOM_STATE)

# Model zur Visualisierung mit attention-output laden
bert_model = TFAutoModelForSequenceClassification.from_pretrained(
    "./bert_model_e2", num_labels=1, output_attentions=True
)
# Compile
bert_model.compile(
    optimizer=tf.keras.optimizers.experimental.Adam(bert_adam_lr), metrics=binary_acc
)

# load and compile vanilla model
bert_model_untrained = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=1, output_attentions=True
)
bert_model_untrained.compile(
    optimizer=tf.keras.optimizers.experimental.Adam(bert_adam_lr), metrics=binary_acc
)


def bert_predict_and_show_attention(
    review: str, use_finetuned_model=True, attention_multiplier: int = 1
):
    """Predicts the sentiment of the given string and uses bert-viz to show attentions."""

    # Tokenize ohne padding
    inputs = tokenizer(
        review,
        return_tensors="np",
        max_length=SEQ_LEN,
        padding=False,
        truncation=True,
    )
    # Vorhersage berechnen
    if use_finetuned_model:
        outputs = bert_model(inputs)
    else:
        outputs = bert_model_untrained(inputs)
    attention = outputs[-1]
    tokens = tokenizer.convert_ids_to_tokens(np.squeeze(inputs["input_ids"]))

    # Konvertierung zu torch-tensor um bert-viz zu nutzen
    torch_attentions = []
    for i in range(len(attention)):
        torch_attentions.append(
            torch.from_numpy(attention[i].numpy()) * attention_multiplier
        )
    torch_attentions_tuple = tuple(torch_attentions)

    # bert-viz anzeigen
    head_view(torch_attentions_tuple, tokens)

    # Ausgabe der Prediction
    print(
        "Predicted Positive Sentiment"
        if tf.keras.activations.sigmoid(outputs[0]).numpy() > 0.5
        else "Predicted Negative Sentiment"
    )

<a id='a3_b'></a>
### 📋 Aufgabe 3 b.

Schauen Sie sich in BertViz für den Satz „This movie was well thought out and badly executed” die Attention-Visualisierung sowohl für das Originalmodell, als auch das 
finetuned-Modell an und probieren Sie ggf. auch andere Beispiele aus: 
1. Starten Sie mit Head X aus Layer 0. Welche Muster erkennt dieser Head wahrscheinlich? Unterscheiden sich Originalmodell und finetuned-Modell?
1. Untersuchen Sie die Attentions für den [CLS] Token aus Head X, Layer 5. Welche Muster erkennt dieser Head wahrscheinlich? Unterscheiden sich Originalmodell und finetuned-Modell?
1. Untersuchen Sie weitere Attention Heads und probieren Sie weitere Beispiele aus. Können Sie weitere Attention-Patterns erkennen?

> **💡 Tipp**\
> Nutzen Sie die Parameter am Anfang der nächsten Zelle um den Output für unterschiedliche Sätze bzw. das finetuned oder original Model zu erhalten.
>

In [ ]:
# Cell-options
sample_idx = 0
attention_multiplier = 3
use_finetuned_model = True

# Beispiel-Texte
sample_texts = [
    """This movie was well thought out and badly executed""",
    """I went to see this movie""",
    """Tony is a really good author and this leads to an excellent and fast movie, which is great but not bad and small. Would watch again!""",
    """The nice movie was played by mediocre actors, but some of them had a beautiful hat with ugly stickers.""",
]

# Bert-Viz und Prediction anzeigen
bert_predict_and_show_attention(
    review=sample_texts[sample_idx],
    use_finetuned_model=use_finetuned_model,
    attention_multiplier=attention_multiplier,
)

<a id='a3_c'></a>
### 📋 Aufgabe 3 c.

Schauen Sie sich die BERT-Vorhersagen für die Top-FNs aus [2b](#a2_b) an – wo funktioniert das Modell besser?

*Ihre Antwort können Sie hier notieren*


***
***
***
***

> # Changelog 
>
> ### v0.6 (14.01.23) 
> - re-added feature relevance for logistic regression
> - suppress warnings for transformers
> - remove length inspection EDA (no relevance for practical)
> - moved comments to end
> - collapsed code
> - removed outputs
>
> ### v0.5 (13.01.23) 
> - fit task descriptions to the pdf file 
> - added js alert telling to restart the kernel after installation 
> - added emojys to main headers (inspired by huggingface)
> - added option to use bertviz on finetuned or original model 
>
> ### v0.4 (11.01.23)
> - adapt to Juypter Lab 
>   - black init settings 
>   - manual restart neccessary after package install 
> - Model metric reduced to accuracy only
> - parameterized embedding-dim in model-create functions
> - print model-metrics as a table
> - bert-viz put into function, so diffent texts can be called without reloading the model 
> - completed comments 
> - removed model-prediction comparison 
> - execution time wihtout install: ~5-6 min 
>
> ### v0.3 (07.02.23)
> - general cleanup 
> - added binary_crossentropy loss to bert model
>
> ### v0.2 (06.02.23)
> - moved utility code to a seperate file 
> - Installing librarys from requirements.txt
> - removed '!nvidia-smi', the assertion in the imports cell should suffice
> - fixed length per sample plot to show words instead of chars
> - model comparison uses standard-scaler instead of min-max-scaler
> - bert-model is saved and reloaded to change wether the attention is returned
> - all models saved with correct seed 
>
>
> ### v0.1 (02.01.23)
> - Added random seed to tensorflow & numpy after importing them
> - Added experimental function call: `tf.config.experimental.enable_op_determinism()`
> - for determinic cells, each training-cell needs to init the optimizer, for this a function is provided that initalizes all compile-arguments on a global level, thus its still apparent to the students that the same options are being used
>     - This function is called: `create_compile_options()`
> - Added random_state argument to sklearn models 
> - Addded accuracy to sklearn models (which is equal to the average precision)
> - removed set_seed function, using transformers.set_seed instead 
> - added misssing torch import 
> - removed example solutions 


---